# Foundry local

<img src="https://learn.microsoft.com/en-us/azure/ai-foundry/foundry-local/media/architecture/foundry-local-arch.png">

Foundry Local is an on-device AI inference solution offering performance, privacy, customization, and cost advantages. It integrates seamlessly into your existing workflows and applications through an intuitive CLI, SDK, and REST API.

## Key features
- On-Device Inference: Run models locally on your own hardware, reducing your costs while keeping all your data on your device.
- Model Customization: Select from preset models or use your own to meet specific requirements and use cases.
- Cost Efficiency: Eliminate recurring cloud service costs by using your existing hardware, making AI more accessible.
- Seamless Integration: Connect with your applications through an SDK, API endpoints, or the CLI, with easy scaling to Azure AI Foundry as your needs grow.

## Use cases
Foundry Local is ideal for scenarios where:
- You want to keep sensitive data on your device.
- You need to operate in environments with limited or no internet connectivity.
- You want to reduce cloud inference costs.
- You need low-latency AI responses for real-time applications.
- You want to experiment with AI models before deploying to a cloud environment.

## Documentation
> https://learn.microsoft.com/en-us/azure/ai-foundry/foundry-local/what-is-foundry-local
> https://github.com/microsoft/Foundry-Local/releases
> https://github.com/microsoft/Foundry-Local/tree/main/docs

## Note

Foundry Local is available in preview. Public preview releases provide early access to features that are in active deployment.
Features, approaches, and processes can change or have limited capabilities, before General Availability (GA).

In [1]:
#%pip install foundry-local-sdk

In [2]:
import json
import openai
import os
import pandas as pd
import requests

from foundry_local import FoundryLocalManager

## List of models

In [3]:
manager = FoundryLocalManager()
manager

In [4]:
manager.is_service_running()

True

In [5]:
manager.service_uri

'http://localhost:5273'

In [6]:
manager.endpoint

'http://localhost:5273/v1'

In [7]:
# List available models in the catalog
catalog = manager.list_catalog_models()
print(f"Available models in the catalog: {catalog}")

Available models in the catalog: [FoundryModelInfo(alias=phi-4, id=Phi-4-cuda-gpu, runtime=cuda, file_size=8570 MB, license=MIT), FoundryModelInfo(alias=phi-4, id=Phi-4-generic-gpu, runtime=webgpu, file_size=8570 MB, license=MIT), FoundryModelInfo(alias=phi-4, id=Phi-4-generic-cpu, runtime=cpu, file_size=10403 MB, license=MIT), FoundryModelInfo(alias=phi-3-mini-128k, id=Phi-3-mini-128k-instruct-cuda-gpu, runtime=cuda, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-3-mini-128k, id=Phi-3-mini-128k-instruct-generic-gpu, runtime=webgpu, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-3-mini-128k, id=Phi-3-mini-128k-instruct-generic-cpu, runtime=cpu, file_size=2600 MB, license=MIT), FoundryModelInfo(alias=phi-3-mini-4k, id=Phi-3-mini-4k-instruct-cuda-gpu, runtime=cuda, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-3-mini-4k, id=Phi-3-mini-4k-instruct-generic-gpu, runtime=webgpu, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-3-mini-4k, id=

In [8]:
for idx, item in enumerate(catalog, start=1):
    print(f"{idx}: {item}\n")

1: alias='phi-4' id='Phi-4-cuda-gpu' version='1' runtime=<ExecutionProvider.CUDA: 'CUDAExecutionProvider'> uri='azureml://registries/azureml/models/Phi-4-cuda-gpu/versions/1' file_size_mb=8570 prompt_template={'system': '<|system|>\n{Content}<|im_end|>', 'user': '<|user|>\n{Content}<|im_end|>', 'assistant': '<|assistant|>\n{Content}<|im_end|>', 'prompt': '<|user|>\n{Content}<|im_end|>\n<|assistant|>'} provider='AzureFoundry' publisher='Microsoft' license='MIT' task='chat-completion'

2: alias='phi-4' id='Phi-4-generic-gpu' version='1' runtime=<ExecutionProvider.WEBGPU: 'WebGpuExecutionProvider'> uri='azureml://registries/azureml/models/Phi-4-generic-gpu/versions/1' file_size_mb=8570 prompt_template={'system': '<|system|>\n{Content}<|im_end|>', 'user': '<|user|>\n{Content}<|im_end|>', 'assistant': '<|assistant|>\n{Content}<|im_end|>', 'prompt': '<|user|>\n{Content}<|im_end|>\n<|assistant|>'} provider='AzureFoundry' publisher='Microsoft' license='MIT' task='chat-completion'

3: alias='ph

In [9]:
data = []

for idx, item in enumerate(catalog, start=1):
    data.append({
        "index": idx,
        "alias": item.alias,
        "id": item.id,
        "version": item.version,
        "runtime": str(item.runtime),
        "uri": item.uri,
        "file_size_mb": item.file_size_mb,
        "prompt_template": item.prompt_template,
        "provider": item.provider,
        "publisher": item.publisher,
        "license": item.license,
        "task": item.task
    })

df = pd.DataFrame(data)
df

,index,alias,id,version,runtime,uri,file_size_mb,prompt_template,provider,publisher,license,task
0,1,phi-4,Phi-4-cuda-gpu,1,CUDAExecutionProvider,azureml://registries/azureml/models/Phi-4-cuda...,8570,"{'system': '<|system|> {Content}<|im_end|>', '...",AzureFoundry,Microsoft,MIT,chat-completion
1,2,phi-4,Phi-4-generic-gpu,1,WebGpuExecutionProvider,azureml://registries/azureml/models/Phi-4-gene...,8570,"{'system': '<|system|> {Content}<|im_end|>', '...",AzureFoundry,Microsoft,MIT,chat-completion
2,3,phi-4,Phi-4-generic-cpu,1,CPUExecutionProvider,azureml://registries/azureml/models/Phi-4-gene...,10403,"{'system': '<|system|> {Content}<|im_end|>', '...",AzureFoundry,Microsoft,MIT,chat-completion
3,4,phi-3-mini-128k,Phi-3-mini-128k-instruct-cuda-gpu,1,CUDAExecutionProvider,azureml://registries/azureml/models/Phi-3-mini...,2181,{'prompt': '<|user|> {Content}<|end|> <|assist...,AzureFoundry,Microsoft,MIT,chat-completion
4,5,phi-3-mini-128k,Phi-3-mini-128k-instruct-generic-gpu,1,WebGpuExecutionProvider,azureml://registries/azureml/models/Phi-3-mini...,2181,"{'system': '<|system|> {Content}<|end|>', 'use...",AzureFoundry,Microsoft,MIT,chat-completion
5,6,phi-3-mini-128k,Phi-3-mini-128k-instruct-generic-cpu,2,CPUExecutionProvider,azureml://registries/azureml/models/Phi-3-mini...,2600,"{'system': '<|system|> {Content}<|end|>', 'use...",AzureFoundry,Microsoft,MIT,chat-completion
6,7,phi-3-mini-4k,Phi-3-mini-4k-instruct-cuda-gpu,1,CUDAExecutionProvider,azureml://registries/azureml/models/Phi-3-mini...,2181,{'prompt': '<|user|> {Content}<|end|> <|assist...,AzureFoundry,Microsoft,MIT,chat-completion
7,8,phi-3-mini-4k,Phi-3-mini-4k-instruct-generic-gpu,1,WebGpuExecutionProvider,azureml://registries/azureml/models/Phi-3-mini...,2181,"{'system': '<|system|> {Content}<|end|>', 'use...",AzureFoundry,Microsoft,MIT,chat-completion
8,9,phi-3-mini-4k,Phi-3-mini-4k-instruct-generic-cpu,2,CPUExecutionProvider,azureml://registries/azureml/models/Phi-3-mini...,2590,"{'system': '<|system|> {Content}<|end|>', 'use...",AzureFoundry,Microsoft,MIT,chat-completion
9,10,mistral-7b-v0.2,mistralai-Mistral-7B-Instruct-v0-2-cuda-gpu,1,CUDAExecutionProvider,azureml://registries/azureml/models/mistralai-...,4075,"{'prompt': '[INST] {Content} [/INST]', 'assist...",AzureFoundry,Microsoft,apache-2.0,chat-completion


In [10]:
for idx in range(len(catalog)):
    model = catalog[idx].id
    print(model)

Phi-4-cuda-gpu
Phi-4-generic-gpu
Phi-4-generic-cpu
Phi-3-mini-128k-instruct-cuda-gpu
Phi-3-mini-128k-instruct-generic-gpu
Phi-3-mini-128k-instruct-generic-cpu
Phi-3-mini-4k-instruct-cuda-gpu
Phi-3-mini-4k-instruct-generic-gpu
Phi-3-mini-4k-instruct-generic-cpu
mistralai-Mistral-7B-Instruct-v0-2-cuda-gpu
mistralai-Mistral-7B-Instruct-v0-2-generic-gpu
mistralai-Mistral-7B-Instruct-v0-2-generic-cpu
Phi-3.5-mini-instruct-cuda-gpu
Phi-3.5-mini-instruct-generic-gpu
Phi-3.5-mini-instruct-generic-cpu
deepseek-r1-distill-qwen-14b-cuda-gpu
deepseek-r1-distill-qwen-14b-generic-gpu
deepseek-r1-distill-qwen-14b-generic-cpu
deepseek-r1-distill-qwen-7b-cuda-gpu
deepseek-r1-distill-qwen-7b-generic-gpu
deepseek-r1-distill-qwen-7b-generic-cpu
qwen2.5-0.5b-instruct-cuda-gpu
qwen2.5-0.5b-instruct-generic-gpu
qwen2.5-0.5b-instruct-generic-cpu
qwen2.5-1.5b-instruct-cuda-gpu
qwen2.5-1.5b-instruct-generic-gpu
qwen2.5-1.5b-instruct-generic-cpu
qwen2.5-coder-7b-instruct-cuda-gpu
qwen2.5-coder-7b-instruct-generi

In [11]:
print("Number of models in the catalog =", len(catalog))

Number of models in the catalog = 50


## Testing

In [12]:
alias = "phi-3.5-mini"

In [13]:
# Download and load a model
model_info = manager.download_model(alias)
model_info = manager.load_model(alias)
print(f"Model info:\n{model_info}")

Model info:
alias='phi-3.5-mini' id='Phi-3.5-mini-instruct-cuda-gpu' version='1' runtime=<ExecutionProvider.CUDA: 'CUDAExecutionProvider'> uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-cuda-gpu/versions/1' file_size_mb=2181 prompt_template={'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>', 'assistant': '<|assistant|>\n{Content}<|end|>'} provider='AzureFoundry' publisher='Microsoft' license='MIT' task='chat-completion'


In [14]:
# List models in cache
local_models = manager.list_cached_models()
print(f"Models in cache:\n{local_models}")

Models in cache:
[FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu, runtime=cuda, file_size=2181 MB, license=MIT)]


In [15]:
# List loaded models
loaded = manager.list_loaded_models()
print(f"Models running in the service:\n{loaded}")

# Unload a model
manager.unload_model(alias)

Models running in the service:
[FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu, runtime=cuda, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-4-mini-reasoning, id=Phi-4-mini-reasoning-generic-cpu, runtime=cpu, file_size=4628 MB, license=MIT)]


In [16]:
# Streaming

alias = "phi-3.5-mini"

manager = FoundryLocalManager(alias)

client = openai.OpenAI(
    base_url=manager.endpoint,
    api_key=manager.api_key  # API key is not required for local usage
)

# Set the model to use and generate a streaming response
stream = client.chat.completions.create(model=manager.get_model_info(alias).id,
                                        messages=[{
                                            "role": "user",
                                            "content": "What is 3.1415?"
                                        }],
                                        stream=True)

# Print the streaming response
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="", flush=True)

 The number 3.1415 is a decimal approximation of the mathematical constant pi (π), which represents the ratio of a circle' extruded circumference to its diameter. Pi is an irrational number, meaning it has an infinite number of non-repeating decimals. The value 3.1415 is accurate to four decimal places, but it is often rounded to 3.14 for simplicity in many practical applications. The more precise value of pi to five decimal places is 3.14159.

In [17]:
# No streaming mode
resp = client.chat.completions.create(
    model=manager.get_model_info(alias).id,
    messages=[{
        "role": "user",
        "content": "What is the capital of France?"
    }],
)

resp

ChatCompletion(id='chat.id.440', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=" The capital of France is Paris. It is not only the country'thy largest city but also serves as the political, cultural, and economic center of the nation. Paris is located in the northern part of France and is home to numerous iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. The city is also known for its significant influence on art, fashion, and cuisine. Paris has a rich history that dates back to the 3rd century and has been an important center for commerce and diplomacy for centuries. The city's status as a capital is further solidified by its role as the seat of the French government and the residence of the President of France.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[], name=None, tool_call_id=None), delta={'role': 'assistant', 'content': " The capital of France is Paris. It 

In [18]:
print(resp.choices[0].message.content)

 The capital of France is Paris. It is not only the country'thy largest city but also serves as the political, cultural, and economic center of the nation. Paris is located in the northern part of France and is home to numerous iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. The city is also known for its significant influence on art, fashion, and cuisine. Paris has a rich history that dates back to the 3rd century and has been an important center for commerce and diplomacy for centuries. The city's status as a capital is further solidified by its role as the seat of the French government and the residence of the President of France.


In [19]:
# List models in cache
local_models = manager.list_cached_models()
print(f"Models in cache:\n{local_models}")

Models in cache:
[FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu, runtime=cuda, file_size=2181 MB, license=MIT)]


In [20]:
print("Number of models in cache =", len(local_models))

Number of models in cache = 1


In [21]:
# List loaded models
loaded = manager.list_loaded_models()
print(f"Models running in the service:\n{loaded}")

# Unload a model
manager.unload_model(alias)

Models running in the service:
[FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu, runtime=cuda, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-4-mini-reasoning, id=Phi-4-mini-reasoning-generic-cpu, runtime=cpu, file_size=4628 MB, license=MIT)]


## Rest API

In [22]:
alias = "mistralai-Mistral-7B-Instruct-v0-2-generic-cpu"

manager = FoundryLocalManager(alias)
url = manager.endpoint + "/chat/completions"

payload = {
    "model": manager.get_model_info(alias).id,
    "messages": [{
        "role": "user",
        "content": "What is Azure?",
    }]
}

headers = {"Content-Type": "application/json"}

response = requests.post(url, headers=headers, data=json.dumps(payload))
print(response.json()["choices"][0]["message"]["content"])

 Azure is a cloud computing platform and set of services offered by Microsoft. It provides a range of solutions including those for computing, storage, networking, databases, analytics, artificial intelligence, and Internet of Things (IoT). Azure enables users to build, deploy, and manage applications and services through a global network of Microsoft-managed data centers. With Azure, businesses and developers can build and run applications with the flexibility of the cloud, while also benefiting from the security, reliability, and scalability of Microsoft's infrastructure.


In [23]:
response.json()

{'model': None,
 'choices': [{'delta': {'role': 'assistant',
    'content': " Azure is a cloud computing platform and set of services offered by Microsoft. It provides a range of solutions including those for computing, storage, networking, databases, analytics, artificial intelligence, and Internet of Things (IoT). Azure enables users to build, deploy, and manage applications and services through a global network of Microsoft-managed data centers. With Azure, businesses and developers can build and run applications with the flexibility of the cloud, while also benefiting from the security, reliability, and scalability of Microsoft's infrastructure.",
    'name': None,
    'tool_call_id': None,
    'function_call': None,
    'tool_calls': []},
   'message': {'role': 'assistant',
    'content': " Azure is a cloud computing platform and set of services offered by Microsoft. It provides a range of solutions including those for computing, storage, networking, databases, analytics, artifici

In [24]:
# List models in cache
local_models = manager.list_cached_models()
print(f"Models in cache:\n{local_models}")

Models in cache:
[FoundryModelInfo(alias=mistral-7b-v0.2, id=mistralai-Mistral-7B-Instruct-v0-2-generic-cpu, runtime=cpu, file_size=4167 MB, license=apache-2.0), FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu, runtime=cuda, file_size=2181 MB, license=MIT)]


In [25]:
for idx in range(len(local_models)):
    print(local_models[idx])
    print()

alias='mistral-7b-v0.2' id='mistralai-Mistral-7B-Instruct-v0-2-generic-cpu' version='2' runtime=<ExecutionProvider.CPU: 'CPUExecutionProvider'> uri='azureml://registries/azureml/models/mistralai-Mistral-7B-Instruct-v0-2-generic-cpu/versions/2' file_size_mb=4167 prompt_template={'system': '<s>', 'user': '[INST]\n{Content}\n[/INST]', 'assistant': '{Content}</s>', 'prompt': '[INST]\n{Content}\n[/INST]'} provider='AzureFoundry' publisher='Microsoft' license='apache-2.0' task='chat-completion'

alias='phi-3.5-mini' id='Phi-3.5-mini-instruct-cuda-gpu' version='1' runtime=<ExecutionProvider.CUDA: 'CUDAExecutionProvider'> uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-cuda-gpu/versions/1' file_size_mb=2181 prompt_template={'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>', 'assistant': '<|assistant|>\n{Content}<|end|>'} provider='AzureFoundry' publisher='Microsoft' license='MIT' task='chat-completion'



In [26]:
print("Number of models in cache =", len(local_models))

Number of models in cache = 2


## CLI

In [27]:
!foundry -h

Description:
  Foundry Local CLI: Run AI models on your device.
  
  ðŸš€ Getting started:
  
     1. To view available models: foundry model list
     2. To run a model: foundry model run <model>
  
     EXAMPLES:
         foundry model run phi-3-mini-4k

UtilisationÂ :
  foundry [command] [options]

OptionsÂ :
  -?, -h, --help  Show help and usage information
  --version       Afficher les informations de version
  --license       Display foudry license information

CommandesÂ :
  model    Discover, run and manage models
  cache    Manage the local cache
  service  Manage the local model inference service



In [28]:
!foundry --version

0.4.91+269dfd9ed1


In [29]:
!foundry model list

Alias                          Device     Task               File Size    License      Model ID            
-----------------------------------------------------------------------------------------------
phi-4                          GPU        chat-completion    8.37 GB      MIT          Phi-4-cuda-gpu      
                               GPU        chat-completion    8.37 GB      MIT          Phi-4-generic-gpu   
                               CPU        chat-completion    10.16 GB     MIT          Phi-4-generic-cpu   
--------------------------------------------------------------------------------------------------------
phi-3-mini-128k                GPU        chat-completion    2.13 GB      MIT          Phi-3-mini-128k-instruct-cuda-gpu
                               GPU        chat-completion    2.13 GB      MIT          Phi-3-mini-128k-instruct-generic-gpu
                               CPU        chat-completion    2.54 GB      MIT          Phi-3-mini-128k-instruct-generic-cp

In [30]:
!foundry model info phi-4-mini-reasoning

Alias                          Device     Task               File Size    License      Model ID            
phi-4-mini-reasoning           GPU        chat-completion    3.15 GB      MIT          Phi-4-mini-reasoning-cuda-gpu


In [31]:
!foundry service restart

In [32]:
!foundry service stop

Restarting service...
ðŸ”´ Service is stopped.
ðŸŸ¢ Service is Started on http://localhost:5273, PID 17624!
ðŸ”´ Service is stopped.
